# 2022-10-17 • General simulator software design

In the previous notebook, the firing rate error in the N-to-1 simulations was fixed. We want to know re-run those simulations with actual lognormal Poisson inputs.

When writing the network simulation code, the N-to-1 simulation code was copied and adapted.
I.e. there is duplication in functionality, and divergence in their APIs.
It's time for consolidation.
Advantage: easier to also investigate LIF/EIF neurons, different neuron types, etc.

## Imports

In [1]:
#

In [2]:
using MyToolbox

In [4]:
@time_imports using VoltoMapSim

## Differential equations

In [16]:
izh = @eqs begin
    
    dv/dt = (k*(v-vₗ)*(v-vₜ) - u - I_syn) / C
    du/dt = a*(b*(v-vᵣ) - u)

    I_syn = gₑ*(v-Eₑ) + gᵢ*(v-Eᵢ)

    dgₑ/dt = -gₑ / τ  # Represents sum over all exc synapses
    dgᵢ/dt = -gᵢ / τ
end;

In [17]:
izh

SpikeLab.Model_
 with variables {v, u, I_syn, gₑ, gᵢ}
 and parameters {C, Eᵢ, Eₑ, a, b, k, vᵣ, vₗ, vₜ, τ}


In [18]:
izh.generated_func

:((diff, vars, params)->begin
          @unpack (v, u, I_syn, gₑ, gᵢ) = vars
          @unpack (C, Eᵢ, Eₑ, a, b, k, vᵣ, vₗ, vₜ, τ) = params
          diff.v = ((k * (v - vₗ) * (v - vₜ) - u) - I_syn) / C
          diff.u = a * (b * (v - vᵣ) - u)
          vars.I_syn = gₑ * (v - Eₑ) + gᵢ * (v - Eᵢ)
          diff.gₑ = -gₑ / τ
          diff.gᵢ = -gᵢ / τ
      end)

## Initialize buffers

In [19]:
params = (
    # Cortical regular spiking (same as always)
    C  =  100    * pF,
    k  =    0.7  * (nS/mV),
    vₗ = - 60    * mV,
    vₜ = - 40    * mV,
    a  =    0.03 / ms,
    b  = -  2    * nS,
    # Not in model eqs above (yet)
    vₛ =   35    * mV,  # spike
    vᵣ = - 50    * mV,  # reset
    Δu =  100    * pA,

    # Synapses
    Eₑ =   0 * mV,
    Eᵢ = -80 * mV,  # Higher than Nto1 (was -65); same as nets.
    τ  =   7 * ms,
)

init = (
    v  = params.vᵣ,
    u  = 0 * pA,
    gₑ = 0 * nS,
    gᵢ = 0 * nS,
    I_syn = 0 * nA,
);

In [20]:
vars = CVec{Float64}(init)
diff = similar(vars)
diff .= 0

ComponentVector{Float64}(v = 0, u = 0, gₑ = 0, gᵢ = 0, I_syn = 0)

In [136]:
]resolve

  No Changes to `C:\Users\tfiers\phd\Project.toml`
  No Changes to `C:\Users\tfiers\phd\Manifest.toml`


In [143]:
Revise.revise(SpikeLab);

In [144]:
SpikeLab.LogNormal(median=3,g=1)

LoadError: UndefVarError: LogNormal not defined

In [ ]:
duration = 1 * second
Δt       = 0.1 * ms


In [125]:
izh.f(diff, vars, params)
diff

ComponentVector{Float64}(v = -0.7, u = -0, gₑ = -0, gᵢ = -0, I_syn = 0)

In [12]:
# Revise.revise(VoltoMapSim);

In [13]:
vars .+= diff * 0.1ms

ComponentVector{Float64}(v = 0.0002, u = -0.0001, I_syn = 8.15E-320, gₑ = 0, gᵢ = 0)

In [14]:
vars

ComponentVector{Float64}(v = 0.0002, u = -0.0001, I_syn = 8.15E-320, gₑ = 0, gᵢ = 0)